In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
# import data from txt
with open("2024\Day_02\data.txt", "r") as f:
    data = f.read().strip().split("\n")
int_data = [list(map(int, re.findall("\\d+", x))) for x in data]

In [ ]:
# Day 2 help from Reddit...
def safe1(levels):
    return all(1 <= abs(n1 - n2) <= 3 for n1, n2 in zip(levels, levels[1:])) and (
        levels == sorted(levels) or levels == sorted(levels)[::-1]
    )


def safe2(levels):
    return any(safe1(levels[:i] + levels[i + 1 :]) for i in range(len(levels)))


# Part 1
print(sum(safe1(levels) for levels in int_data))

# Part 2
print(sum(safe2(levels) for levels in int_data))

In [ ]:
import re

# Example patterns
pattern1 = r"start"
pattern2 = r"middle"
pattern3 = r"reset"

# Example string
text = "start some text middle reset start middle start reset middle"

# Compile patterns
p1 = re.compile(pattern1)
p2 = re.compile(pattern2)
p3 = re.compile(pattern3)

# Initialize flags and results
search_for_p2 = False
results = []

# Iterate through all matches in the string
for match in re.finditer(rf"{pattern1}|{pattern2}|{pattern3}", text):
    match_text = match.group()

    if p1.fullmatch(match_text):  # Found pattern1
        search_for_p2 = True
    elif p3.fullmatch(match_text):  # Found pattern3
        search_for_p2 = False
    elif p2.fullmatch(match_text) and search_for_p2:  # Found pattern2 while searching for it
        results.append(match_text)
        search_for_p2 = False  # Stop searching for pattern2 until next pattern1

# Output results
print("Pattern2 Matches:", results)


In [ ]:
def find_words_multiple(grid, words):
    rows = len(grid)
    cols = len(grid[0])
    
    # Define 8 possible directions (row_offset, col_offset)
    directions = [
        (0, 1),   # Right
        (0, -1),  # Left
        (1, 0),   # Down
        (-1, 0),  # Up
        (1, 1),   # Down-Right
        (-1, -1), # Up-Left
        (1, -1),  # Down-Left
        (-1, 1),  # Up-Right
    ]
    
    def is_valid(x, y):
        """Check if coordinates are within bounds."""
        return 0 <= x < rows and 0 <= y < cols
    
    def search_from(x, y, word):
        """Search for a word starting from (x, y) in all directions."""
        positions = []
        for row_offset, col_offset in directions:
            nx, ny = x, y
            match = True
            for char in word:
                if not is_valid(nx, ny) or grid[nx][ny] != char:
                    match = False
                    break
                nx += row_offset
                ny += col_offset
            if match:
                # If a match is found, record the start position and direction
                positions.append((x, y, row_offset, col_offset))
        return positions
    
    # Result dictionary
    found_words = {word: [] for word in words}
    
    # Search for each word
    for word in words:
        for i in range(rows):
            for j in range(cols):
                if grid[i][j] == word[0]:
                    occurrences = search_from(i, j, word)
                    if occurrences:
                        found_words[word].extend(occurrences)
    
    return found_words

# Example word search grid
word_search = [
    ['c', 'a', 't', 'f'],
    ['b', 'g', 'd', 'o'],
    ['x', 'y', 'z', 'g'],
    ['d', 'o', 'g', 'a']
]

# Words to find
word_list = ["cat", "dog", "god", "fat", "xyz"]

# Solve word search
results = find_words_multiple(word_search, word_list)

# Print results
for word, positions in results.items():
    if positions:
        print(f"Word '{word}' found {len(positions)} times at positions: {positions}")
    else:
        print(f"Word '{word}' not found")


In [ ]:
def find_diagonal_x_for_word(grid, word):
    rows = len(grid)
    cols = len(grid[0])

    # Define diagonal directions
    directions = [
        (1, 1),   # Down-Right
        (-1, -1), # Up-Left
        (1, -1),  # Down-Left
        (-1, 1),  # Up-Right
    ]

    def is_valid(x, y):
        """Check if coordinates are within bounds."""
        return 0 <= x < rows and 0 <= y < cols

    def search_from(x, y, word, direction):
        """Search for a word starting from (x, y) in a single diagonal direction."""
        row_offset, col_offset = direction
        nx, ny = x, y
        path = []
        for char in word:
            if not is_valid(nx, ny) or grid[nx][ny] != char:
                return None  # Invalid path
            path.append((nx, ny))
            nx += row_offset
            ny += col_offset
        return path

    def check_x_shape(path1, path2):
        """Check if two paths form an 'X' by intersecting at their midpoints."""
        mid1 = path1[len(path1) // 2]  # Middle of first path
        mid2 = path2[len(path2) // 2]  # Middle of second path
        return mid1 == mid2  # Intersection at the same point

    # Search the grid for the word forming an 'X'
    x_shapes = []
    for i in range(rows):
        for j in range(cols):
            if grid[i][j] == word[0]:  # Starting point must match the first letter
                # Search in all diagonal directions
                for dir1 in directions:
                    path1 = search_from(i, j, word, dir1)
                    if path1:
                        # For the second diagonal, pick the opposite direction
                        for dir2 in directions:
                            if dir1 == dir2 or dir1 == (-dir2[0], -dir2[1]):
                                continue  # Skip same or reverse directions
                            path2 = search_from(i, j, word, dir2)
                            if path2 and check_x_shape(path1, path2):
                                x_shapes.append((path1, path2))

    return x_shapes

# Example word search grid
word_search = [
    ['M', 'M', 'M', 'S', 'X', 'X', 'M', 'A', 'S', 'M'],
    ['M', 'S', 'A', 'M', 'X', 'M', 'S', 'M', 'S', 'A'],
    ['A', 'M', 'X', 'S', 'X', 'M', 'A', 'A', 'M', 'M'],
    ['M', 'S', 'A', 'M', 'A', 'S', 'M', 'S', 'M', 'X'],
    ['X', 'M', 'A', 'S', 'A', 'M', 'X', 'A', 'M', 'M'],
    ['X', 'X', 'A', 'M', 'M', 'X', 'X', 'A', 'M', 'A'],
    ['S', 'M', 'S', 'M', 'S', 'A', 'S', 'X', 'S', 'S'],
    ['S', 'A', 'X', 'A', 'M', 'A', 'S', 'A', 'A', 'A'],
    ['M', 'A', 'M', 'M', 'M', 'X', 'M', 'M', 'M', 'M'],
    ['M', 'X', 'M', 'X', 'A', 'X', 'M', 'A', 'S', 'X'],
]

# Word to find
word = "MAS"

# Solve word search for 'X' shape
results = find_diagonal_x_for_word(word_search, word)

# Print results
if results:
    for paths in results:
        path1, path2 = paths
        print(f"Word '{word}' forms an 'X' at intersection with paths:")
        print(f"  Path1: {path1}")
       


In [ ]:
def find_diagonal_x_for_word(grid, word):
    rows = len(grid)
    cols = len(grid[0])

    # Define diagonal directions
    directions = [
        (1, 1),   # Down-Right
        (-1, -1), # Up-Left
        (1, -1),  # Down-Left
        (-1, 1),  # Up-Right
    ]

    def is_valid(x, y):
        """Check if coordinates are within bounds."""
        return 0 <= x < rows and 0 <= y < cols

    def search_from(x, y, word, direction):
        """Search for a word starting from (x, y) in a single diagonal direction."""
        row_offset, col_offset = direction
        nx, ny = x, y
        path = []
        for char in word:
            if not is_valid(nx, ny) or grid[nx][ny] != char:
                return None  # Invalid path
            path.append((nx, ny))
            nx += row_offset
            ny += col_offset
        return path

    def verify_x_shape(paths):
        """Verify if any two paths form an 'X' by intersecting at their midpoints."""
        midpoints = {tuple(path[len(path) // 2]) for path in paths}  # Extract all midpoints
        # Check if any midpoint is shared between two paths
        result = []
        for i in range(len(paths)):
            for j in range(i + 1, len(paths)):
                mid1 = paths[i][len(paths[i]) // 2]
                mid2 = paths[j][len(paths[j]) // 2]
                if mid1 == mid2:  # Shared midpoint
                    result.append((paths[i], paths[j]))
        return result

    # Search for all diagonal occurrences of the word
    all_paths = []
    for i in range(rows):
        for j in range(cols):
            if grid[i][j] == word[0]:  # Starting point must match the first letter
                for direction in directions:
                    path = search_from(i, j, word, direction)
                    if path:
                        all_paths.append(path)

    # Verify which paths form 'X' shapes
    x_shapes = verify_x_shape(all_paths)
    return x_shapes


# Example word search grid
word_search = [
    ['M', 'M', 'M', 'S', 'X', 'X', 'M', 'A', 'S', 'M'],
    ['M', 'S', 'A', 'M', 'X', 'M', 'S', 'M', 'S', 'A'],
    ['A', 'M', 'X', 'S', 'X', 'M', 'A', 'A', 'M', 'M'],
    ['M', 'S', 'A', 'M', 'A', 'S', 'M', 'S', 'M', 'X'],
    ['X', 'M', 'A', 'S', 'A', 'M', 'X', 'A', 'M', 'M'],
    ['X', 'X', 'A', 'M', 'M', 'X', 'X', 'A', 'M', 'A'],
    ['S', 'M', 'S', 'M', 'S', 'A', 'S', 'X', 'S', 'S'],
    ['S', 'A', 'X', 'A', 'M', 'A', 'S', 'A', 'A', 'A'],
    ['M', 'A', 'M', 'M', 'M', 'X', 'M', 'M', 'M', 'M'],
    ['M', 'X', 'M', 'X', 'A', 'X', 'M', 'A', 'S', 'X'],
]

# Word to find
word = "MAS"

# Solve word search for 'X' shape
results = find_diagonal_x_for_word(word_search, word)

# Print results
if results:
    print(f"\nTotal 'X' shapes found: {len(results)}")
    for paths in results:
        path1, path2 = paths
        print(f"Word '{word}' forms an 'X' at intersection with paths:")
        print(f"  Path1: {path1}")
        print(f"  Path2: {path2}")    
else:
    print(f"Word '{word}' does not form an 'X' in the grid.")


In [ ]:
from collections import defaultdict, deque

def validate_order(rules, order):
    # Build a graph and calculate in-degrees
    graph = defaultdict(list)
    in_degree = defaultdict(int)
    
    # Build the graph from rules
    for a, b in rules:
        graph[a].append(b)
        in_degree[b] += 1
        in_degree.setdefault(a, 0)  # Ensure all nodes are in the in-degree map
    
    # Perform topological sort using Kahn's algorithm
    queue = deque([node for node in in_degree if in_degree[node] == 0])
    topological_order = []
    
    while queue:
        current = queue.popleft()
        topological_order.append(current)
        for neighbor in graph[current]:
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                queue.append(neighbor)
    
    # If the graph has a cycle, the topological order won't include all nodes
    if len(topological_order) != len(in_degree):
        raise ValueError("The rules contain a cycle, so no valid order exists.")
    
    # Check if the input order respects the topological order
    position = {node: i for i, node in enumerate(topological_order)}
    for i in range(len(order) - 1):
        if position[order[i]] > position[order[i + 1]]:
            return False  # Order is invalid
    
    return True

# Example usage
rules = [('A', 'B'), ('B', 'C'), ('C', 'D')]
order = ['A', 'B', 'C', 'D']  # Valid order
invalid_order = ['A', 'C', 'B', 'D']  # Invalid order

print(validate_order(rules, order))         # Output: True
print(validate_order(rules, invalid_order))  # Output: False


In [18]:
import numpy as np
import pandas as pd
from io import StringIO

# Read the file into memory
with open('2024\\Day_05\\test_data.txt', 'r') as file:
    sections = file.read().strip().split("\n\n")
print(sections)
# Parse each section using pandas
data1 = np.loadtxt(sections[0].splitlines(), delimiter="|").tolist()
data2 = np.loadtxt(sections[1].splitlines(), delimiter=",").tolist()

print("Data Set 1:")
print(data1)

print("Data Set 2:")
print(data2)


['47|53\n97|13\n97|61\n97|47\n75|29\n61|13\n75|53\n29|13\n97|29\n53|29\n61|53\n97|53\n61|29\n47|13\n75|47\n97|75\n47|61\n75|61\n47|29\n75|13\n53|13', '75,47,61,53,29\n97,61,53,29,13\n75,29,13\n75,97,47,61,53\n61,13,29\n97,13,75,29,47']


ValueError: the number of columns changed from 5 to 3 at row 3; use `usecols` to select a subset and avoid this error